In [1]:
import keras
from keras import callbacks
from keras.datasets import mnist
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
# out = np.load('out.npy')
# c = np.load('c.npy')
# l = np.load('l.npy')
# q = np.load('q.npy')
a = np.load('a.npy')

In [4]:
# a_int = [0,0,1,1,2,0,3,3,3,1]
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['garden', 'garden', 'hallway', 'hallway', 'kitchen', 'garden', 'bedroom',
 'bedroom', 'bedroom', 'hallway'])
train_labels = le.transform(['garden', 'garden', 'hallway', 'hallway', 'kitchen', 'garden', 'bedroom',
 'bedroom', 'bedroom', 'hallway'])

In [5]:
train_labels

array([1, 1, 2, 2, 3, 1, 0, 0, 0, 2], dtype=int64)

In [6]:
one_hot_labels = keras.utils.to_categorical(train_labels, num_classes=4)
print (one_hot_labels)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [7]:
RN_input_small = np.load('RN_input_small.npy')

In [8]:
RN_input_small.shape

(10, 190, 1536)

In [9]:
#  def g_theta(self, RN_input, scope='g_theta', reuse = True, phase = True):
#         """
#         Args
#             RN_input: [o_i, o_j, q], shape = [batch_size*190, 136]
#         Returns
#             g_output: shape = [190, batch_size, 256]
#         """
#         input_dim = RN_input.shape[1]
#         g_units = [256,256,256,256]
#         with tf.variable_scope(scope, reuse = reuse) as scope:
#             g_1 = self.batch_norm_relu(RN_input, g_units[0], scope= 'g_1', phase = phase)
#             g_2 = self.batch_norm_relu(g_1, g_units[1], scope='g_2', phase=phase)
#             g_3 = self.batch_norm_relu(g_2, g_units[2], scope='g_3', phase=phase)
#             g_4 = self.batch_norm_relu(g_3, g_units[3], scope='g_4', phase=phase)
#         g_output = tf.reshape(g_4, shape= [-1, self.batch_size, g_units[3]])
#         return g_output

In [10]:
def g_theta(RN_input, batch_size = 1):
    """
    
    """
    input_shape = RN_input.shape[2]
    g_units = [256,256,256,256]
    
    inputs = Input((input_shape,))
    g1 = Dense(g_units[0], activation = 'relu')(inputs)
    g2 = Dense(g_units[1], activation = 'relu')(g1)
    g3 = Dense(g_units[2], activation = 'relu')(g2)
    g4 = Dense(g_units[3], activation = 'relu')(g3)
    g_out = Reshape((-1, batch_size, g_units[3]))(g4)
    g_sum = K.sum(g_out, axis = 0)    
    
    return g_sum

In [11]:
def f_phi(g_sum):
    """
    f_units[2] is set to be 6 as there are only 6 unique answers for 
    task 2.
    """
    f_units = [256,512,6]
    
    f1 = Dense(f_units[0], activation = 'relu')(g_sum)
    f2 = Dense(f_units[1], activation = 'relu')(f1)
    f3 = Dense(f_units[2], activation = 'relu')(f2)
    
    return f3   

In [12]:
def build(RN_input):
    """
    """
    f_input = g_theta(RN_input)
    pred = f_phi(f_input)
    
    return pred


In [13]:
RN_input_small.shape[1:3]

(190, 1536)

In [28]:
batch_size = 1

input_shape = RN_input_small.shape[1:3]
g_units = [256,256,256,256]

inputs = Input((input_shape))
g1 = Dense(g_units[0], activation = 'relu')(inputs)
g2 = Dense(g_units[1], activation = 'relu')(g1)
g3 = Dense(g_units[2], activation = 'relu')(g2)
g4 = Dense(g_units[3], activation = 'relu')(g3)
# g_out = Reshape((-1, batch_size, g_units[3]))(g4)
# g_sum = Lambda(lambda x: K.sum(x, axis=1))(g_out)


f_units = [256,512,4] # for first 10 samples of the RN_input

f1 = Dense(f_units[0], activation = 'relu')(g_sum)
f2 = Dense(f_units[1], activation = 'relu')(f1)
f3 = Dense(f_units[2], activation = 'softmax')(f2)
f3 = Flatten()(f3)
    

In [29]:
model = Model(input = inputs, output = f3)

C:\Users\zlai\AppData\Local\Continuum\anaconda3\envs\rn\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fl...)`
  """Entry point for launching an IPython kernel.


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 190, 1536)         0         
_________________________________________________________________
dense_22 (Dense)             (None, 190, 256)          393472    
_________________________________________________________________
dense_23 (Dense)             (None, 190, 256)          65792     
_________________________________________________________________
dense_24 (Dense)             (None, 190, 256)          65792     
_________________________________________________________________
dense_25 (Dense)             (None, 190, 256)          65792     
_________________________________________________________________
reshape_4 (Reshape)          (None, 190, 1, 256)       0         
_________________________________________________________________
lambda_4 (Lambda)            (None, 1, 256)            0         
__________

In [22]:
model.compile(optimizer = Adam(lr = 2e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [23]:
# model_checkpoints = callbacks.ModelCheckpoint("weights_{epoch:02d}_{val_loss:.2f}.h5", monitor='val_loss',
#                                                     verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [24]:
model.fit(x = RN_input_small, 
          y = one_hot_labels, 
          epochs=5, 
          validation_split= 0.1, 
#           callbacks=[model_checkpoints]
         )

Train on 9 samples, validate on 1 samples
Epoch 1/5
9/9 [==============================] - 2s 249ms/step - loss: 1.6091 - acc: 0.2222 - val_loss: 1.2817 - val_acc: 0.0000e+00
Epoch 2/5
9/9 [==============================] - 0s 17ms/step - loss: 1.1940 - acc: 0.5556 - val_loss: 1.0963 - val_acc: 0.0000e+00
Epoch 3/5
9/9 [==============================] - 0s 18ms/step - loss: 1.0252 - acc: 0.4444 - val_loss: 0.7195 - val_acc: 1.0000
Epoch 4/5
9/9 [==============================] - 0s 17ms/step - loss: 0.8780 - acc: 0.6667 - val_loss: 0.5274 - val_acc: 1.0000
Epoch 5/5
9/9 [==============================] - 0s 17ms/step - loss: 0.7729 - acc: 0.8889 - val_loss: 0.3719 - val_acc: 1.0000
